In [1]:
Threads.nthreads()

1

In [4]:
using POMDPs, QuickPOMDPs, Statistics, CSV, DataFrames, JLD2, NPZ

In [3]:
using Pkg
Pkg.develop(path="D:\\Repos\\welfare-maximization-cdc\\simulations\\ParticleFilters")

   Resolving package versions...
  No Changes to `C:\Users\pranayt2\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\pranayt2\.julia\environments\v1.8\Manifest.toml`


In [4]:
using ParticleFilters

[ Info: Precompiling ParticleFilters [c8b314e2-9260-5cf8-ae76-3be7461ca6d0]


In [5]:
using Pkg
Pkg.develop(path="D:\\Repos\\welfare-maximization-cdc\\simulations\\POMDPTools")

   Resolving package versions...
  No Changes to `C:\Users\pranayt2\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\pranayt2\.julia\environments\v1.8\Manifest.toml`


In [6]:
using POMDPTools

In [7]:
pathof(ParticleFilters)

"D:\\Repos\\welfare-maximization-cdc\\simulations\\ParticleFilters\\src\\ParticleFilters.jl"

In [8]:
include("temps/BasicPOMCP_2.jl")

┌ Warning: POMDPs.@implemented is deprecated, use POMDPLinter.@implemented instead.
└ @ POMDPs C:\Users\pranayt2\.julia\packages\POMDPs\XBTe5\src\deprecated.jl:10


Main.BasicPOMCP

In [14]:
function save_data(filename::AbstractString, data)
    jldopen(filename, "w") do file
        file["data"] = data
    end
end

function build_pomdp(component_id, replacement_cost, inspection_cost, budget)
    num_trials = 5;
    horizon = 100;
    component_id = string(component_id)

    # num_trials x horizon sized array for each logging element
    state_histories = Array{Tuple{Int64,Int64},2}(undef, num_trials, horizon);
    action_histories = Array{String,2}(undef,, num_trials, horizon);
    observation_histories = Array{Tuple{Int64,Int64},2}(undef, num_trials, horizon);
    reward_histories = Array{Float64,2}(undef, num_trials, horizon);

    # println("Running simulation for component = "*component_id)
    dynamics = []
    open("dynamics/dynamics_"*component_id*".csv") do f
        line = 0  
        while ! eof(f) 
            # read a new / next line for every iteration          
            s = readline(f)
            s = parse.(Float64, split(chop(s; head=1, tail=1), ','))
            push!(dynamics,s)
            line += 1
        end
    end

    buildingprob = QuickPOMDP(
            actions = ["maintain", "inspect", "do-nothing"],

            transition = function(s,a)
                next_states_inspect = []
                next_states_nothing = []
                for i in 0:s[1]
                    push!(next_states_inspect, (i,s[2]+inspection_cost))
                    push!(next_states_nothing, (i,s[2]))
                end
                if a == "maintain"
                    return Deterministic((100, s[2] + replacement_cost))
                elseif a == "inspect"
                    return SparseCat(next_states_inspect, dynamics[s[1]+1])
                elseif a == "do-nothing"
                    return SparseCat(next_states_nothing, dynamics[s[1]+1])
                end
            end,

            observation = function(s,a,sp)
                if a == "inspect"
                    return Deterministic(sp)
                elseif a == "maintain"
                    return Deterministic(sp)
                elseif a == "do-nothing"
                    return Deterministic((101, sp[2]))
                end
            end,

            reward = function(s,a,sp)
                if sp[1] > 0
                    return 1
                else
                    return 0
                end
            end,

            obstype = Tuple{Int64,Int64},
            actiontype = String,
            initialstate = Deterministic((100, 0)),
            isterminal = function(s)
                if s[2] + replacement_cost > budgets[budget] || s[1] == 0
                    return true
                else
                    return false
                end
            end)

    solver_q = BasicPOMCP.POMCPSolver(max_depth=60, c=10, tree_queries=4000);
    planner_q = solve(solver_q, buildingprob);

    for trial in 1:num_trials
        count = 1
        for (s, a, o, r, b) in stepthrough(buildingprob, planner_q, "s,a,o,r,b", max_steps = horizon)
            state_histories[trial, count] = s
            action_histories[trial, count] = a
            observation_histories[trial, count] = o
            reward_histories[trial, count] = r
            count += 1
        end
    end

    save_data("results/allocated_runs/state_histories_"*component_id*".jld2", state_histories)
    save_data("results/allocated_runs/action_histories_"*component_id*".jld2", action_histories)
    save_data("results/allocated_runs/observation_histories_"*component_id*".jld2", observation_histories)
    save_data("results/allocated_runs/reward_histories_test_"*component_id*".jld2", reward_histories)
end

build_pomdp (generic function with 1 method)

In [15]:
function parallel_cpu!(x,y,z)
    Threads.@threads for i in 1:length(x)
        build_pomdp(x[i],y[i],z[i], budgets[i])
    end
    return nothing
end

df = DataFrames.DataFrame(CSV.File("selected_15_components_data.csv"))
component_ids = df[!,"component_id"]
replacement_costs = df[!,"replacement_cost"]
inspection_costs = df[!,"inspection_cost"]
budgets = npzread("proposed_allocated_budgets.npy")

# specify which indices to run (indexed by rows order in selected_15_components_data NOT by component id)
indices_to_run = [15]  #[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

parallel_cpu!(component_ids[indices_to_run],replacement_costs[indices_to_run],inspection_costs[indices_to_run], budgets[indices_to_run])